## Implementing TimeSeriesRandomForestClassifier
Open sklearn `RandomForestClassifier` to accept pipelines as `base_estimator`, this required to 
* change how parameters are set in construction of individual member of ensemble 
* pass on random state from top-level to all components of pipeline with randomness, including modifying `Pipeline` to have random_state attribute
* adapt sklearn helper functions for parallel building and accumulating predictions of trees
* remove sklearn input checks for tabular data, ideally replace with input checks for time-series/panel data, possibly implemented as part of datacontainer, include keyword argument to sidestep input checks in members of ensembles similar to check_input in sklearn `DecisionTreeClassifier`

Still to do:
* write unit test
* `feature_importances_` has to be removed or adapted to a temporal importance curve (as in the paper)
* `decision_path()` has to be adapted
* `oob_score` methods have to be adapted
* `apply` method has to be adapted
* speed up transfoms, still very slow

In [5]:
from sktime.tsforest import TimeSeriesForest
from sktime.load_data import load_from_web_to_xdataframe
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
cache_path = 'data/'
dataset_name = 'GunPoint'
X_train, y_train = load_from_web_to_xdataframe(dataset_name, is_train_file=True,
                                               cache_path=cache_path) 
X_test, y_test = load_from_web_to_xdataframe(dataset_name, is_test_file=True,
                                             cache_path=cache_path)
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)
X_test = pd.DataFrame(X_test)
y_test = pd.Series(y_test)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(50, 1) (50,) (150, 1) (150,)


In [7]:
clf = TimeSeriesForest(n_estimators=2, criterion='entropy', random_state=123)
%time clf.fit(X_train, y_train)
%time clf.score(X_test, y_test)

CPU times: user 3.84 s, sys: 10.1 ms, total: 3.85 s
Wall time: 3.84 s
CPU times: user 10.8 s, sys: 34.5 ms, total: 10.9 s
Wall time: 10.9 s


0.9133333333333333